# python notebook for robust estimation of the mean of a vector via a simple search algorithm in the mutual distance matrix of empirical means on blocks

This notebook has been used to performed the simulation study in ...

In [3]:
import numpy as np
from math import floor, exp, log
from numpy import sqrt
from numpy.random import multivariate_normal, randn
import math
from sklearn.utils import shuffle
#import warnings
#warnings.filterwarnings("ignore")

np.set_printoptions(precision=2)

import matplotlib.pylab as plt


%matplotlib inline

## define various types of outliers

In [4]:
def data1(n_samples, n_features, cov):    
    X = multivariate_normal(np.zeros(n_features), cov, size=n_samples)
    return X

def data2(n_outliers, n_features, type_outliers = 1):
    if type_outliers == 1:
        X = 1000*np.ones((n_outliers, n_features))
    elif type_outliers == 2:
        X = np.random.rand(n_outliers, n_features)
    else:
        mean = np.ones(n_features)
        cov = np.identity(n_features)
        X = multivariate_normal(mean, cov, size=n_samples)
    return X

def data3(n_heavy_tail, n_features, deg = 2):
    X = np.random.standard_t(deg, size=(n_heavy_tail, n_features))
    return X

## Merge datasets

In [5]:
def data_merge_with_outlier_info(X1, X2, X3):
    X = np.concatenate((X1, X2, X3), axis=0)
    #outlier1p = np.concatenate((np.full(X1.size,False), np.full(X2.size,True), np.full(X3.size, False)))
    #outlier2p = np.concatenate((np.full(X1.size,False), np.full(X2.size,False), np.full(X3.size, True)))
    outlier1p = np.concatenate((np.full(X1.shape[0],False), np.full(X2.shape[0],True), np.full(X3.shape[0], False)))
    outlier2p = np.concatenate((np.full(X1.shape[0],False), np.full(X2.shape[0],False), np.full(X3.shape[0], True)))
    return shuffle(X, outlier1p, outlier2p)


def mom_generate_data(n_samples, n_outliers, n_heavy_tail, n_features, cov):
    X1 = data1(n_samples, n_features,  cov)
    X2 = data2(n_outliers, n_features, type_outliers = 1)
    X3 = data3(n_heavy_tail, n_features, deg = 2.5)
    X, outlier1p, outlier2p = data_merge_with_outlier_info(X1, X2, X3)

    X = np.matrix(X)
    return X, outlier1p, outlier2p

## Construct a dataset

In [6]:
n_samples, n_outliers, n_heavy_tail, n_features = 100000, 0, 0, 50
cov = np.identity(n_features)

In [7]:
X, outlier1p, outlier2p = mom_generate_data(n_samples, n_outliers, n_heavy_tail, n_features, cov)

In [8]:
print(X[23])

[[-1.08  0.55  0.06  0.59  0.52  2.18 -1.44 -1.86 -1.46 -1.06 -0.78 -0.23
   1.57  0.36  0.51 -0.12  1.18  0.33 -1.71  0.33  0.55 -1.64  1.1  -0.03
  -1.99  0.75  0.16  0.53  1.25  0.27 -0.37  0.71  0.77 -0.8   0.74  0.11
  -1.75  0.76 -0.12 -0.37 -0.08  0.24  0.78 -0.05 -0.53  0.05  0.2   0.07
  -0.66  0.73]]


## Construction of the $K$ empirical means

In [9]:
def construction_means(X, K):
    list_K_means = []
    for k in range(K):
        m=floor(X.shape[0]/K)
        list_K_means.append(np.array(np.mean(X[k*m:(k+1)*m, :], axis=0))[0].tolist())
    return list_K_means

In [10]:
L=construction_means(X,100000)

In [11]:
print((L[1]))

[0.8231488935993743, -0.5798619808398165, -0.8252029736301498, -0.3788669086907916, 0.14272264609353058, -1.0567897469055496, 0.12154880323970413, 0.8001498038298094, 0.4208462997311291, 0.5900230280407646, 0.957180009192976, -1.2051240360571218, 1.1904013883111777, 0.48735078267431553, 0.0021903290872088066, -1.1465735436597806, 0.33049933696955835, -0.19215331722230958, -0.4161698941439009, -0.5435584612465952, 0.6081151916200805, 0.5012650258353303, -0.20608357425736532, 0.6573910325878105, -1.366807560256276, 0.35506052484623324, 0.9982490225901274, -0.3732642161080864, -1.039713484688698, 0.26067774017579226, 0.7212079227226319, -0.7735747914611487, 0.7271520940266842, -0.2556379609263368, -1.5160862366157861, 1.239230117883599, -0.07897975978417875, 0.17855282140057027, 1.1136913749055142, 0.32490790768786504, -0.020718070892154255, -0.40694975441594256, -0.46298826713491437, 0.7555382399864018, 0.6806447623538331, -0.21519039016511796, 0.4867494671452752, 0.5166398277673789, 2.0

## Mutual distance matrix construction

In [12]:
def mutual_table (X,K,dist):
    list_K_means=construction_means(X, K)
    M=np.zeros((K, K))
    for j in range(K):
        for i in range(j+1):
            M[i,j]=dist(list_K_means[i],list_K_means[j])
            M[j,i]=dist(list_K_means[i],list_K_means[j])
    return M
    

In [13]:
def eucl_dist(A,B):
    s=0
    for i in range(len(A)):
        s=s+(A[i]-B[i])**2
    return sqrt(s)

In [14]:
print(eucl_dist([-3,0,0],[1,1,2]))

4.58257569495584


In [15]:
mutual_table(X,1000,eucl_dist)

array([[0.  , 0.88, 0.96, ..., 0.98, 0.96, 0.83],
       [0.88, 0.  , 0.92, ..., 0.98, 1.  , 1.07],
       [0.96, 0.92, 0.  , ..., 1.1 , 1.07, 1.02],
       ...,
       [0.98, 0.98, 1.1 , ..., 0.  , 0.95, 1.2 ],
       [0.96, 1.  , 1.07, ..., 0.95, 0.  , 1.05],
       [0.83, 1.07, 1.02, ..., 1.2 , 1.05, 0.  ]])

## Looking for the smallest median

In [16]:
def Choose_point(X,K, dist):
    list_med=construction_means(X, K)
    M=mutual_table (X,K,dist)
    val=np.median(M[0])
    i_0=0
    for i in range(K):
        if np.median(M[i])<val:
            val=np.median(M[i])
            i_0=i
    #print(i_0)
    return list_med[i_0]

In [17]:
Choose_point(X,1000,eucl_dist)

[-0.035758395977580064,
 -0.034977571317024066,
 0.028768984561540773,
 -0.13927791833667835,
 -0.041150305349484245,
 0.07619164821972117,
 -0.15362830188747284,
 -0.03637153586398428,
 -0.00961897122349619,
 0.05453553010308574,
 0.0843505451265453,
 -0.009489891348429438,
 -0.06294590408641679,
 -0.00858717672854343,
 0.11224969050018635,
 -0.05771393407563446,
 0.03336973860998915,
 0.01094151680308644,
 0.08462133486144328,
 0.03432634015087152,
 0.03027311103274391,
 -0.007423529147900212,
 -0.09488496333455329,
 -0.002828921949869598,
 -0.0015636006107701572,
 -0.08058623124549563,
 0.12287180441351647,
 -0.02605211132782697,
 -0.024905121598044356,
 0.02092608119661992,
 0.04773549133295693,
 -0.03400193002196783,
 0.0981253810460875,
 0.050161722478033466,
 -0.05882694633173788,
 0.0939704539722693,
 -0.00474825732196452,
 -0.009520490930213857,
 0.10013209678083557,
 -0.013298704563250894,
 -0.0021257637000343865,
 0.07676986244493335,
 -0.03667232445059001,
 0.02879449366667

In [18]:
eucl_dist(Choose_point(X,27,eucl_dist),np.zeros(50))

0.08857952439942439

# Autosélection de K

In [19]:
def Autoselect_k_v1(X,confidence_intervals, dist): 
    N=len(X)
    K=N
    m=1
    point_list=[]
    K_list=[]
    condition=True
    condition_2=True
    while condition and condition_2:
        if K>sqrt(N): #on fait évoluer la manière dont K change
            current=Choose_point(X,K,dist)
            i=0
            for point in point_list:
                condition= (condition and dist(point,current)<confidence_intervals[K_list[i]-1])
                i=i+1
            if K==1:
                condition_2=False
            if condition:
                point_list.append(current)
                K_list.append(K)
                m=m+1
                K=floor(len(X)/m)
        else: 
            current=Choose_point(X,K,dist)
            i=0
            for point in point_list:
                #print(dist(point,current))
                condition= (condition and dist(point,current)<confidence_intervals[K_list[i]-1])
                i=i+1
            if K==1:
                condition_2=False
            if condition:
                point_list.append(current)
                K_list.append(K)
                K=K-1
    print(K_list[-1])
    return K_list[-1] #,point_list[-1]
    

In [20]:
confidence_intervalles=[]
N=len(X)
for i in range(N):
    confidence_intervalles.append(2.9*np.sqrt(n_features*(i+1))/sqrt(N))
print(confidence_intervalles)

[0.0648459713474939, 0.09170605214488299, 0.11231651704001508, 0.1296919426949878, 0.14499999999999996, 0.15883954167649816, 0.17156631370988884, 0.18341210428976598, 0.1945379140424817, 0.20506096654409875, 0.2150697561257742, 0.22463303408003016, 0.2338054747006579, 0.2426314076948819, 0.2511473670974872, 0.2593838853899756, 0.2673667892614937, 0.275118156434649, 0.2826570359994599, 0.2899999999999999, 0.2971615722128283, 0.304154565969344, 0.31099035354814464, 0.3176790833529963, 0.3242298567374695, 0.33065087327875, 0.33694955112004527, 0.3431326274197777, 0.34920624278497653, 0.3551760127035608, 0.3610470883416732, 0.36682420857953196, 0.3725117447812887, 0.3781137395017536, 0.38363394010436563, 0.3890758280849634, 0.39444264475332785, 0.3997374138106164, 0.40496296126929926, 0.4101219330881975, 0.41521681083501416, 0.4202499256394936, 0.4252234706598402, 0.4301395122515484, 0.43499999999999994, 0.4398067757549899, 0.4445615817859208, 0.4492660681600603, 0.45392179943245725, 0.458

In [21]:
log(2)

0.6931471805599453

In [22]:
K_pour_20=[]
for i in range(10):
    X, outlier1p, outlier2p = mom_generate_data(n_samples, n_outliers, n_heavy_tail, n_features, cov)
    K_pour_20.append(Autoselect_k_v1(X,confidence_intervalles,eucl_dist))
print(K_pour_20)

KeyboardInterrupt: 

In [ ]:
np.mean(K_pour_20)

In [ ]:
30/log(2)

In [23]:
len([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],[11, 7, 10, 8, 8, 11, 4, 6, 6, 11, 9, 10, 5, 11, 11, 11, 4, 11, 7, 10],[11, 20, 17, 15, 14, 17, 11, 17, 10, 18, 11, 13, 16, 8, 11, 10, 15, 16, 8, 18],[19, 16, 24, 21, 27, 21, 22, 23, 19, 16, 17, 22, 23, 23, 25, 17, 23, 17, 28, 24],[22, 29, 33, 24, 33, 30, 32, 34, 30, 24, 28, 11, 20, 22, 20, 19, 35, 28, 27, 27],[40, 44, 40, 29, 25, 34, 37, 29, 31, 28, 41, 24, 32, 40, 19, 33, 35, 36, 25, 35],[38, 42, 28, 40, 41, 46, 28, 41, 34, 39, 36, 39, 36, 42, 41, 39, 45, 37, 49, 42],[45, 48, 46, 31, 31, 46, 49, 43, 41, 32, 52, 24, 34, 50, 50, 33, 41, 55, 34, 53],[61, 44, 64, 56, 56, 47, 61, 69, 51, 52, 60, 41, 58, 49, 60, 46, 39, 53, 46, 26],[67, 72, 58, 40, 51, 54, 60, 44, 39, 60, 54, 61, 57, 69, 48, 70, 57, 45, 43, 52],[68, 53, 74, 57, 58, 69, 62, 62, 71, 76, 69, 62, 73, 69, 73, 52, 78, 43, 70, 73],[77, 68, 74, 74, 81, 74, 81, 88, 66, 65, 49, 82, 76, 66, 86, 79, 56, 75, 68, 79],[77, 89, 87, 83, 83, 76, 74, 91, 78, 86, 84, 68, 86, 81, 91, 52, 81, 77, 86, 72],[86, 81, 84, 90, 90, 88, 76, 67, 95, 88, 92, 92, 89, 80, 86, 97, 102, 90, 83, 74],[86, 95, 93, 90, 71, 94, 99, 101, 90, 79, 87, 82, 89, 100, 76, 93, 96, 94, 91, 106],[110, 100, 110, 76, 81, 77, 100, 109, 96, 87, 101, 74, 84, 72, 87, 96, 93, 98, 96, 76],[103, 110, 107, 109, 106, 122, 115, 94, 100, 91, 114, 105, 100, 105, 107, 99, 100, 121, 105, 96],[98, 127, 120, 99, 112, 68, 114, 120, 108, 98, 117, 121, 117, 114, 112, 110, 118, 121, 120, 121],[138, 117, 109, 129, 98, 120, 126, 115, 109, 140, 90, 101, 121, 104, 118, 134, 126, 100, 121, 124],[87, 105, 114, 131, 105, 91, 107, 112, 97, 138, 96, 98, 142, 132, 129, 116, 127, 108, 110, 112], [124, 138, 129, 109, 97, 121, 129, 82, 142, 138, 123, 142, 127, 134, 118, 123, 127, 129, 113, 120],[146, 132, 146, 120, 136, 142, 116, 116, 160, 106, 134, 127, 108, 132, 123, 146, 138, 118, 120, 121], [134, 142, 97, 155, 102, 129, 138, 150, 127, 133, 155, 146, 157, 168, 168, 150, 150, 171, 163, 155], [112, 160, 134, 140, 131, 150, 160, 133, 144, 121, 133, 146, 146, 104, 133, 163, 131, 138, 148, 134]])

24

In [28]:
np.apply_along_axis(np.min,1,[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],[11, 7, 10, 8, 8, 11, 4, 6, 6, 11, 9, 10, 5, 11, 11, 11, 4, 11, 7, 10],[11, 20, 17, 15, 14, 17, 11, 17, 10, 18, 11, 13, 16, 8, 11, 10, 15, 16, 8, 18],[19, 16, 24, 21, 27, 21, 22, 23, 19, 16, 17, 22, 23, 23, 25, 17, 23, 17, 28, 24],[22, 29, 33, 24, 33, 30, 32, 34, 30, 24, 28, 11, 20, 22, 20, 19, 35, 28, 27, 27],[40, 44, 40, 29, 25, 34, 37, 29, 31, 28, 41, 24, 32, 40, 19, 33, 35, 36, 25, 35],[38, 42, 28, 40, 41, 46, 28, 41, 34, 39, 36, 39, 36, 42, 41, 39, 45, 37, 49, 42],[45, 48, 46, 31, 31, 46, 49, 43, 41, 32, 52, 24, 34, 50, 50, 33, 41, 55, 34, 53],[61, 44, 64, 56, 56, 47, 61, 69, 51, 52, 60, 41, 58, 49, 60, 46, 39, 53, 46, 26],[67, 72, 58, 40, 51, 54, 60, 44, 39, 60, 54, 61, 57, 69, 48, 70, 57, 45, 43, 52],[68, 53, 74, 57, 58, 69, 62, 62, 71, 76, 69, 62, 73, 69, 73, 52, 78, 43, 70, 73],[77, 68, 74, 74, 81, 74, 81, 88, 66, 65, 49, 82, 76, 66, 86, 79, 56, 75, 68, 79],[77, 89, 87, 83, 83, 76, 74, 91, 78, 86, 84, 68, 86, 81, 91, 52, 81, 77, 86, 72],[86, 81, 84, 90, 90, 88, 76, 67, 95, 88, 92, 92, 89, 80, 86, 97, 102, 90, 83, 74],[86, 95, 93, 90, 71, 94, 99, 101, 90, 79, 87, 82, 89, 100, 76, 93, 96, 94, 91, 106],[110, 100, 110, 76, 81, 77, 100, 109, 96, 87, 101, 74, 84, 72, 87, 96, 93, 98, 96, 76],[103, 110, 107, 109, 106, 122, 115, 94, 100, 91, 114, 105, 100, 105, 107, 99, 100, 121, 105, 96],[98, 127, 120, 99, 112, 68, 114, 120, 108, 98, 117, 121, 117, 114, 112, 110, 118, 121, 120, 121],[138, 117, 109, 129, 98, 120, 126, 115, 109, 140, 90, 101, 121, 104, 118, 134, 126, 100, 121, 124],[87, 105, 114, 131, 105, 91, 107, 112, 97, 138, 96, 98, 142, 132, 129, 116, 127, 108, 110, 112], [124, 138, 129, 109, 97, 121, 129, 82, 142, 138, 123, 142, 127, 134, 118, 123, 127, 129, 113, 120],[146, 132, 146, 120, 136, 142, 116, 116, 160, 106, 134, 127, 108, 132, 123, 146, 138, 118, 120, 121], [134, 142, 97, 155, 102, 129, 138, 150, 127, 133, 155, 146, 157, 168, 168, 150, 150, 171, 163, 155], [112, 160, 134, 140, 131, 150, 160, 133, 144, 121, 133, 146, 146, 104, 133, 163, 131, 138, 148, 134]])

array([  1,   4,   8,  16,  11,  19,  28,  24,  26,  39,  43,  49,  52,
        67,  71,  72,  91,  68,  90,  87,  82, 106,  97, 104])

In [29]:
np.apply_along_axis(np.max,1,[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],[11, 7, 10, 8, 8, 11, 4, 6, 6, 11, 9, 10, 5, 11, 11, 11, 4, 11, 7, 10],[11, 20, 17, 15, 14, 17, 11, 17, 10, 18, 11, 13, 16, 8, 11, 10, 15, 16, 8, 18],[19, 16, 24, 21, 27, 21, 22, 23, 19, 16, 17, 22, 23, 23, 25, 17, 23, 17, 28, 24],[22, 29, 33, 24, 33, 30, 32, 34, 30, 24, 28, 11, 20, 22, 20, 19, 35, 28, 27, 27],[40, 44, 40, 29, 25, 34, 37, 29, 31, 28, 41, 24, 32, 40, 19, 33, 35, 36, 25, 35],[38, 42, 28, 40, 41, 46, 28, 41, 34, 39, 36, 39, 36, 42, 41, 39, 45, 37, 49, 42],[45, 48, 46, 31, 31, 46, 49, 43, 41, 32, 52, 24, 34, 50, 50, 33, 41, 55, 34, 53],[61, 44, 64, 56, 56, 47, 61, 69, 51, 52, 60, 41, 58, 49, 60, 46, 39, 53, 46, 26],[67, 72, 58, 40, 51, 54, 60, 44, 39, 60, 54, 61, 57, 69, 48, 70, 57, 45, 43, 52],[68, 53, 74, 57, 58, 69, 62, 62, 71, 76, 69, 62, 73, 69, 73, 52, 78, 43, 70, 73],[77, 68, 74, 74, 81, 74, 81, 88, 66, 65, 49, 82, 76, 66, 86, 79, 56, 75, 68, 79],[77, 89, 87, 83, 83, 76, 74, 91, 78, 86, 84, 68, 86, 81, 91, 52, 81, 77, 86, 72],[86, 81, 84, 90, 90, 88, 76, 67, 95, 88, 92, 92, 89, 80, 86, 97, 102, 90, 83, 74],[86, 95, 93, 90, 71, 94, 99, 101, 90, 79, 87, 82, 89, 100, 76, 93, 96, 94, 91, 106],[110, 100, 110, 76, 81, 77, 100, 109, 96, 87, 101, 74, 84, 72, 87, 96, 93, 98, 96, 76],[103, 110, 107, 109, 106, 122, 115, 94, 100, 91, 114, 105, 100, 105, 107, 99, 100, 121, 105, 96],[98, 127, 120, 99, 112, 68, 114, 120, 108, 98, 117, 121, 117, 114, 112, 110, 118, 121, 120, 121],[138, 117, 109, 129, 98, 120, 126, 115, 109, 140, 90, 101, 121, 104, 118, 134, 126, 100, 121, 124],[87, 105, 114, 131, 105, 91, 107, 112, 97, 138, 96, 98, 142, 132, 129, 116, 127, 108, 110, 112], [124, 138, 129, 109, 97, 121, 129, 82, 142, 138, 123, 142, 127, 134, 118, 123, 127, 129, 113, 120],[146, 132, 146, 120, 136, 142, 116, 116, 160, 106, 134, 127, 108, 132, 123, 146, 138, 118, 120, 121], [134, 142, 97, 155, 102, 129, 138, 150, 127, 133, 155, 146, 157, 168, 168, 150, 150, 171, 163, 155], [112, 160, 134, 140, 131, 150, 160, 133, 144, 121, 133, 146, 146, 104, 133, 163, 131, 138, 148, 134]])

array([  1,  11,  20,  28,  35,  44,  49,  55,  69,  72,  78,  88,  91,
       102, 106, 110, 122, 127, 140, 142, 142, 160, 171, 163])

In [30]:
np.apply_along_axis(np.mean,1,[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],[11, 7, 10, 8, 8, 11, 4, 6, 6, 11, 9, 10, 5, 11, 11, 11, 4, 11, 7, 10],[11, 20, 17, 15, 14, 17, 11, 17, 10, 18, 11, 13, 16, 8, 11, 10, 15, 16, 8, 18],[19, 16, 24, 21, 27, 21, 22, 23, 19, 16, 17, 22, 23, 23, 25, 17, 23, 17, 28, 24],[22, 29, 33, 24, 33, 30, 32, 34, 30, 24, 28, 11, 20, 22, 20, 19, 35, 28, 27, 27],[40, 44, 40, 29, 25, 34, 37, 29, 31, 28, 41, 24, 32, 40, 19, 33, 35, 36, 25, 35],[38, 42, 28, 40, 41, 46, 28, 41, 34, 39, 36, 39, 36, 42, 41, 39, 45, 37, 49, 42],[45, 48, 46, 31, 31, 46, 49, 43, 41, 32, 52, 24, 34, 50, 50, 33, 41, 55, 34, 53],[61, 44, 64, 56, 56, 47, 61, 69, 51, 52, 60, 41, 58, 49, 60, 46, 39, 53, 46, 26],[67, 72, 58, 40, 51, 54, 60, 44, 39, 60, 54, 61, 57, 69, 48, 70, 57, 45, 43, 52],[68, 53, 74, 57, 58, 69, 62, 62, 71, 76, 69, 62, 73, 69, 73, 52, 78, 43, 70, 73],[77, 68, 74, 74, 81, 74, 81, 88, 66, 65, 49, 82, 76, 66, 86, 79, 56, 75, 68, 79],[77, 89, 87, 83, 83, 76, 74, 91, 78, 86, 84, 68, 86, 81, 91, 52, 81, 77, 86, 72],[86, 81, 84, 90, 90, 88, 76, 67, 95, 88, 92, 92, 89, 80, 86, 97, 102, 90, 83, 74],[86, 95, 93, 90, 71, 94, 99, 101, 90, 79, 87, 82, 89, 100, 76, 93, 96, 94, 91, 106],[110, 100, 110, 76, 81, 77, 100, 109, 96, 87, 101, 74, 84, 72, 87, 96, 93, 98, 96, 76],[103, 110, 107, 109, 106, 122, 115, 94, 100, 91, 114, 105, 100, 105, 107, 99, 100, 121, 105, 96],[98, 127, 120, 99, 112, 68, 114, 120, 108, 98, 117, 121, 117, 114, 112, 110, 118, 121, 120, 121],[138, 117, 109, 129, 98, 120, 126, 115, 109, 140, 90, 101, 121, 104, 118, 134, 126, 100, 121, 124],[87, 105, 114, 131, 105, 91, 107, 112, 97, 138, 96, 98, 142, 132, 129, 116, 127, 108, 110, 112], [124, 138, 129, 109, 97, 121, 129, 82, 142, 138, 123, 142, 127, 134, 118, 123, 127, 129, 113, 120],[146, 132, 146, 120, 136, 142, 116, 116, 160, 106, 134, 127, 108, 132, 123, 146, 138, 118, 120, 121], [134, 142, 97, 155, 102, 129, 138, 150, 127, 133, 155, 146, 157, 168, 168, 150, 150, 171, 163, 155], [112, 160, 134, 140, 131, 150, 160, 133, 144, 121, 133, 146, 146, 104, 133, 163, 131, 138, 148, 134]])

array([  1.  ,   8.55,  13.8 ,  21.35,  26.4 ,  32.85,  39.15,  41.9 ,
        51.95,  55.05,  65.6 ,  73.2 ,  80.1 ,  86.5 ,  90.6 ,  91.15,
       105.45, 111.75, 117.  , 112.85, 123.25, 129.35, 144.5 , 138.05])